In [ ]:
### Running questions
### How to handle Ns? [We're probably going to ignore them]
### Should I remove reference from alternate the same way I remove alternate from the reference? 
### Find longest string rather than checking entire stretch and neighbors to see if they're homopolymers? 

### Note: I will clean up stuff for sure (moving lines to functions, etc), this is more about logic 

In [ ]:
import cyvcf2
from pyfaidx import Fasta
import os
import argparse

In [ ]:
os.chdir('/Users/quinlan/u1264408/Git/spermseq')
os.listdir()

In [ ]:
A = '1SCS014.indels.vcf'
Atrust = '1SCS014.trustworthyindels.vcf'
B = 'K8J7003.indels.vcf'
C = 'UA6U012.indels.vcf'
D = 'YL99029.indels.vcf'
E = 'XH9R013.indels.vcf'

In [ ]:
genome = Fasta('/Users/quinlan/u1264408/human_g1k_v38_decoy_phix.fasta', as_raw=True) # look into as_RAW
# as_raw: optional parameter to specify whether to return sequences as a Sequence() object or as a raw string.
from cyvcf2 import VCF

In [ ]:
### 1 
### GOOD TO GO  
### Simplest code: is alternate allele in reference genome sequence twice the length of alternate allele

# vcf = cyvcf2.VCF(sys.argv[2])
vcf = VCF(Atrust)

failcount = 0
passcount = 0 

for v in vcf():
    #lenalt = max(len(v.ALT[0]), len(v.REF))
    lenalt = len(v.ALT[0])
    genome_seq_stretch = genome[v.CHROM][v.POS-lenalt:v.POS+lenalt]
    if 'N' in v.ALT[0]:
        #print('FAIL ', v.ALT[0])
        failcount += 1 
    elif v.ALT[0] in genome_seq_stretch:
        #print('FAIL ', v.CHROM, v.POS, v.REF, v.ALT[0])
        #print(genome_seq_stretch)
        failcount += 1 
    else:
        #print('PASS', v.CHROM, v.POS, v.REF, v.ALT[0])
        #print(genome_seq_stretch)
        passcount += 1
print(v.INFO.get('donor') + ":", 'Passcount is', passcount, 'and failcount is', failcount, ".", "Total is", (passcount + failcount)) 

In [ ]:
### 2
###  GOOD TO GO (I THINK)
### More complex: let's think about allele size between reference and alternate alleles 
## Insertions: alt allele > ref allele
## Version where there's repeat? Or I remove ref allele? 
## Look for alt allele in reference genome 
## Deletions: ref allele > alt allele 
## Is alt allele contained within reference allele? Is the string difference in the left or right stretch?

# vcf = cyvcf2.VCF(sys.argv[2])
vcf = VCF(Atrust)

failcount = 0
passcount = 0 

for v in vcf():
    lenalt = max(len(v.ALT[0]), len(v.REF))
    genome_seq_left = genome[v.CHROM][v.POS-lenalt:v.POS]
    genome_seq_right = genome[v.CHROM][v.POS:v.POS+lenalt]
    genome_seq_stretch = genome[v.CHROM][v.POS-lenalt:v.POS+lenalt]
    if 'N' in v.ALT[0]:
        print('FAIL ', v.ALT[0])
        failcount += 1 
    elif len(v.ALT[0]) > len(v.REF) and v.ALT[0] in genome_seq_stretch:
        print('FAIL ', v.CHROM, v.POS, v.REF, v.ALT[0])
        print(genome_seq_left, genome_seq_right, genome_seq_stretch)
        failcount += 1 
    elif len(v.ALT[0]) < len(v.REF):
        valt = v.ALT[0]
        vref = v.REF
        if valt in vref:
            vref = vref.replace(valt, '', 1)
            #only replace once 
            if vref in genome_seq_left or genome_seq_right:
                print('FAIL ', v.CHROM, v.POS, v.REF, v.ALT[0])
                print(genome_seq_left, genome_seq_right, genome_seq_stretch)
                failcount += 1 
            else: 
                print('PASS ', v.CHROM, v.POS, v.REF, v.ALT[0])
                print(genome_seq_left, genome_seq_right, genome_seq_stretch)
                passcount += 1
        else:
            print('PASS ', v.CHROM, v.POS, v.REF, v.ALT[0])
            print(genome_seq_left, genome_seq_right, genome_seq_stretch)    
            passcount += 1
    else:
        print('PASS', v.CHROM, v.POS, v.REF, v.ALT[0])
        print(genome_seq_left, genome_seq_right, genome_seq_stretch)
        passcount += 1
print(v.INFO.get('donor') + ":", 'Passcount is', passcount, 'and failcount is', failcount) 
   


In [ ]:
### 3
###  Probably good to go 
### Less strict: checks for larger alt alleles in genome stretch and only homopolymers fail with regards to reference allele in adjacent stretches
vcf = VCF(Atrust)

failcount = 0
passcount = 0 

for v in vcf():
    lenalt = max(len(v.ALT[0]), len(v.REF))
    genome_seq_left = genome[v.CHROM][v.POS-lenalt:v.POS]
    genome_seq_right = genome[v.CHROM][v.POS:v.POS+lenalt]
    
    if (genome_seq_left == len(genome_seq_left) * genome_seq_left[0]):
        genome_seq_left_use = genome_seq_left
    else:
        genome_seq_left_use = 'no_left'
    genome_seq_right = genome[v.CHROM][v.POS:v.POS+lenalt]
    if (genome_seq_right == len(genome_seq_right) * genome_seq_right[0]):
        genome_seq_right_use = genome_seq_right
    else:
        genome_seq_right_use = 'no_right'
    genome_seq_stretch = genome[v.CHROM][v.POS-lenalt:v.POS+lenalt]

    if 'N' in v.ALT[0]:
        print('FAIL ', v.ALT[0])
        failcount += 1 
    elif len(v.ALT[0]) > len(v.REF) and v.ALT[0] in genome_seq_stretch:
        print('FAIL in stretch', v.CHROM, v.POS, v.REF, v.ALT[0])
        print(genome_seq_left, genome_seq_right, genome_seq_stretch)
        failcount += 1 
    elif (len(v.ALT[0]) > len(v.REF)) and (v.ALT[0] not in genome_seq_stretch):
        print('PASS should not be in stretch', v.CHROM, v.POS, v.REF, v.ALT[0])
        print(genome_seq_left, genome_seq_right, genome_seq_stretch)
        passcount += 1
    elif len(v.ALT[0]) < len(v.REF):
        valt = v.ALT[0]
        vref = v.REF
        if vref in valt:
            vref = vref.replace(valt, '', 1)
            if vref in genome_seq_left_use or genome_seq_right_use:
                print('FAIL right or left', v.CHROM, v.POS, v.REF, v.ALT[0])
                print(genome_seq_left, genome_seq_right, genome_seq_stretch)
                failcount += 1 
            else: 
                print('PASS not in left or right', v.CHROM, v.POS, v.REF, v.ALT[0])
                print(genome_seq_left, genome_seq_right, genome_seq_stretch)
                passcount += 1

        else:
            print('PASS leftover', v.CHROM, v.POS, v.REF, v.ALT[0])
            print(genome_seq_left, genome_seq_right, genome_seq_stretch)    
            passcount += 1
    else:
        print('PASS leftover', v.CHROM, v.POS, v.REF, v.ALT[0])
        print(genome_seq_left, genome_seq_right, genome_seq_stretch)
        passcount += 1
print(v.INFO.get('donor') + ":", 'Passcount is', passcount, 'and failcount is', failcount) 
   


In [ ]:
### 4, fine but could be fiddled with 
### Least strict: only homopolymers fail with regards to reference allele in adjacent stretches AND alternate alleles in homopolymers
vcf = VCF(Atrust)

failcount = 0
passcount = 0 

for v in vcf():
    lenalt = max(len(v.ALT[0]), len(v.REF))
    genome_seq_left = genome[v.CHROM][v.POS-lenalt:v.POS]
    genome_seq_right = genome[v.CHROM][v.POS:v.POS+lenalt]
    genome_seq_stretch = genome[v.CHROM][v.POS-lenalt:v.POS+lenalt]
    
    if (genome_seq_left == len(genome_seq_left) * genome_seq_left[0]):
        genome_seq_left_use = genome_seq_left
    else:
        genome_seq_left_use = 'Left'
    genome_seq_right = genome[v.CHROM][v.POS:v.POS+lenalt]
    if (genome_seq_right == len(genome_seq_right) * genome_seq_right[0]):
        genome_seq_right_use = genome_seq_right
    else:
        genome_seq_right_use = 'Right'
    genome_seq_stretch = genome[v.CHROM][v.POS-lenalt:v.POS+lenalt]
    if (genome_seq_stretch == len(genome_seq_stretch) * genome_seq_stretch[0]):
        genome_seq_stretch_use = genome_seq_stretch
    else:
        genome_seq_stretch_use = 'Stretch'
    valt = v.ALT[0]
    vref = v.REF
    if len(v.ALT[0]) < len(v.REF):
        vref = vref.replace(valt, '', 1)
    elif len(v.ALT[0]) > len(v.REF):
        valt = valt.replace(vref, '', 1)
    else:
        pass

    if 'N' in v.ALT[0]:
        print('FAIL ', v.ALT[0])
        failcount += 1 
    elif len(v.ALT[0]) > len(v.REF) and valt in genome_seq_stretch_use:
        #print('FAIL in stretch', v.CHROM, v.POS, v.REF, v.ALT[0])
        #print(genome_seq_left, genome_seq_right, genome_seq_stretch)
        failcount += 1 
    elif (len(v.ALT[0]) > len(v.REF)) and (valt not in genome_seq_stretch_use):
        #print('PASS not in stretch/stretch not homopolymer', v.CHROM, v.POS, v.REF, v.ALT[0])
        #print(genome_seq_left, genome_seq_right, genome_seq_stretch)
        passcount += 1
    elif len(v.ALT[0]) < len(v.REF):
        if vref in valt:
            vref = vref.replace(valt, '', 1)
            if vref in genome_seq_left_use or genome_seq_right_use:
                #print('FAIL right or left', v.CHROM, v.POS, v.REF, v.ALT[0])
                #print(genome_seq_left, genome_seq_right, genome_seq_stretch)
                failcount += 1 
            else: 
                #print('PASS not in left or right', v.CHROM, v.POS, v.REF, v.ALT[0])
                #print(genome_seq_left, genome_seq_right, genome_seq_stretch)
                passcount += 1
        else:
            #print('PASS leftover', v.CHROM, v.POS, v.REF, v.ALT[0])
            #print(genome_seq_left, genome_seq_right, genome_seq_stretch)    
            passcount += 1
    else:
        #print('PASS leftover', v.CHROM, v.POS, v.REF, v.ALT[0])
        #print(genome_seq_left, genome_seq_right, genome_seq_stretch)
        passcount += 1
print(v.INFO.get('donor') + ":", 'Passcount is', passcount, 'and failcount is', failcount) 
   


In [ ]:
def Substring(s):
 
    ans, temp = 1, 1
 
    # Traverse the string
    for i in range(1, len(s)):
         
        # If character is same as
        # previous increment temp value
        if (s[i] == s[i - 1]):
            temp += 1
        else:
            ans = max(ans, temp)
            temp = 1
 
    ans = max(ans, temp)
 
    # Return the required answer
    return ans
 
# Driver code
s = "abcdddddeff"
 
# Function call
print(Substring(s))
# Mohit Kumar 

In [ ]:
# Strategy 5: a modified strategy 3
# taking only the difference between v.alt and v.ref in analysis

vcf = VCF(A)

failcount = 0
passcount = 0 

for v in vcf():
    lenalt = max(len(v.ALT[0]), len(v.REF))
    genome_seq_left = genome[v.CHROM][v.POS-lenalt:v.POS]
    genome_seq_right = genome[v.CHROM][v.POS:v.POS+lenalt]
    
    if (genome_seq_left == len(genome_seq_left) * genome_seq_left[0]):
        genome_seq_left_use = genome_seq_left
    else:
        genome_seq_left_use = 'no_left'
    genome_seq_right = genome[v.CHROM][v.POS:v.POS+lenalt]
    if (genome_seq_right == len(genome_seq_right) * genome_seq_right[0]):
        genome_seq_right_use = genome_seq_right
    else:
        genome_seq_right_use = 'no_right'
    genome_seq_stretch = genome[v.CHROM][v.POS-lenalt:v.POS+lenalt]

    valt = v.ALT[0]
    vref = v.REF
    if len(vref) > len(valt):
        vref = vref.replace(valt, '', 1)
    else: 
        valt = valt.replace(vref, '', 1)

    if 'N' in v.ALT[0]:
        print('FAIL ', v.ALT[0])
        failcount += 1 
    elif len(v.ALT[0]) > len(v.REF) and valt in genome_seq_stretch:
        print('FAIL in stretch', v.CHROM, v.POS, v.REF, v.ALT[0])
        print(genome_seq_left, genome_seq_right, genome_seq_stretch)
        failcount += 1 
    elif (len(v.ALT[0]) > len(v.REF)) and (valt not in genome_seq_stretch):
        print('PASS should not be in stretch', v.CHROM, v.POS, v.REF, v.ALT[0])
        print(genome_seq_left, genome_seq_right, genome_seq_stretch)
        passcount += 1
    elif len(v.ALT[0]) < len(v.REF):
        if vref in genome_seq_left_use or genome_seq_right_use:
            print('FAIL right or left', v.CHROM, v.POS, v.REF, v.ALT[0])
            print(genome_seq_left, genome_seq_right, genome_seq_stretch)                
            failcount += 1 
        else: 
            print('PASS not in left or right', v.CHROM, v.POS, v.REF, v.ALT[0])
            print(genome_seq_left, genome_seq_right, genome_seq_stretch)                
            passcount += 1
         
    else:
        print('PASS leftover', v.CHROM, v.POS, v.REF, v.ALT[0])
        print(genome_seq_left, genome_seq_right, genome_seq_stretch)
        passcount += 1
print(v.INFO.get('donor') + ":", 'Passcount is', passcount, 'and failcount is', failcount) 
   


In [ ]:
### 3 BUT in left and right instead of stretch
###  Probably good to go 
### Less strict: checks for larger alt alleles in genome stretch and only homopolymers fail with regards to reference allele in adjacent stretches
vcf = VCF(Atrust)

failcount = 0
passcount = 0 

for v in vcf():
    lenalt = max(len(v.ALT[0]), len(v.REF))
    genome_seq_left = genome[v.CHROM][v.POS-lenalt:v.POS]
    genome_seq_right = genome[v.CHROM][v.POS:v.POS+lenalt]
    
    if (genome_seq_left == len(genome_seq_left) * genome_seq_left[0]):
        genome_seq_left_use = genome_seq_left
    else:
        genome_seq_left_use = 'no_left'
    genome_seq_right = genome[v.CHROM][v.POS:v.POS+lenalt]
    if (genome_seq_right == len(genome_seq_right) * genome_seq_right[0]):
        genome_seq_right_use = genome_seq_right
    else:
        genome_seq_right_use = 'no_right'
    genome_seq_stretch = genome[v.CHROM][v.POS-lenalt:v.POS+lenalt]
    
    valt = v.ALT[0]
    vref = v.REF
    if len(vref) > len(valt):
        vref = vref.replace(valt, '', 1)
    else: 
        valt = valt.replace(vref, '', 1)
    if 'N' in v.ALT[0]:
        print('FAIL ', v.ALT[0])
        failcount += 1 
    elif len(v.ALT[0]) > len(v.REF) and (valt in (genome_seq_left or genome_seq_right)):
        print('FAIL in stretch', v.CHROM, v.POS, v.REF, v.ALT[0])
        print(genome_seq_left, genome_seq_right, genome_seq_stretch)
        failcount += 1 
    elif (len(v.ALT[0]) > len(v.REF)) and (valt not in (genome_seq_left or genome_seq_right)):
        print('PASS should not be in stretch', v.CHROM, v.POS, v.REF, v.ALT[0])
        print(genome_seq_left, genome_seq_right, genome_seq_stretch)
        passcount += 1
    elif len(v.ALT[0]) < len(v.REF):
        valt = v.ALT[0]
        vref = v.REF
        if vref in valt:
            vref = vref.replace(valt, '', 1)
            if vref in genome_seq_left_use or genome_seq_right_use:
                print('FAIL right or left', v.CHROM, v.POS, v.REF, v.ALT[0])
                print(genome_seq_left, genome_seq_right, genome_seq_stretch)
                failcount += 1 
            else: 
                print('PASS not in left or right', v.CHROM, v.POS, v.REF, v.ALT[0])
                print(genome_seq_left, genome_seq_right, genome_seq_stretch)
                passcount += 1

        else:
            print('PASS leftover', v.CHROM, v.POS, v.REF, v.ALT[0])
            print(genome_seq_left, genome_seq_right, genome_seq_stretch)    
            passcount += 1
    else:
        print('PASS leftover', v.CHROM, v.POS, v.REF, v.ALT[0])
        print(genome_seq_left, genome_seq_right, genome_seq_stretch)
        passcount += 1
print(v.INFO.get('donor') + ":", 'Passcount is', passcount, 'and failcount is', failcount) 
   


In [ ]:
def Substring(s):
    ans, temp = 1, 1
    # Traverse the string
    for i in range(1, len(s)):
        # If character is same as
        # previous increment temp value
        if (s[i] == s[i - 1]):
            temp += 1
        else:
            ans = max(ans, temp)
            temp = 1
 
        ans = max(ans, temp)
    # the required answer
    return ans

vcf = VCF(Atrust)

failcount = 0
passcount = 0 

for v in vcf():
    genome_seq_left = genome[v.CHROM][v.POS-10:v.POS]
    genome_seq_right = genome[v.CHROM][v.POS:v.POS+10]
    
    valt = v.ALT[0]
    vref = v.REF
    if len(vref) > len(valt):
        vdiff = vref.replace(valt, '', 1)
    else: 
        vdiff = valt.replace(vref, '', 1)
    
    sleft = (Substring(genome_seq_left)
    vlen = (len(vdiff))

    print(Substring(genome_seq_left), genome_seq_left, Substring(genome_seq_right), genome_seq_right)
    if (vdiff * )) in genome_seq_left:
        print( v.CHROM, v.POS, v.REF, v.ALT[0])            
        failcount += 1
        print('vdiff in left')
    elif (vdiff * (Substring(genome_seq_right)/len(vdiff))) in genome_seq_right: 
        print( v.CHROM, v.POS, v.REF, v.ALT[0])
        failcount += 1
        print('vdiff in right')
    else:
        print( v.CHROM, v.POS, v.REF, v.ALT[0])
        passcount += 1
        print('PASS')
print(v.INFO.get('donor') + ":", 'Passcount is', passcount, 'and failcount is', failcount) 

 

In [88]:
### Not touching 5

def Substring(s):
    ans, temp = 1, 1
    # Traverse the string
    for i in range(1, len(s)):
        # If character is same as
        # previous increment temp value
        if (s[i] == s[i - 1]):
            temp += 1
        else:
            ans = max(ans, temp)
            temp = 1
 
        ans = max(ans, temp)
    # the required answer
    return ans

vcf = VCF(Atrust)

failcount = 0
passcount = 0 

for v in vcf():
    genome_seq_left = genome[v.CHROM][v.POS-12:v.POS]
    genome_seq_right = genome[v.CHROM][v.POS:v.POS+12]
    
    valt = v.ALT[0]
    vref = v.REF
    if len(vref) > len(valt):
        vdiff = vref.replace(valt, '', 1)
    else: 
        vdiff = valt.replace(vref, '', 1)
    
    print(Substring(genome_seq_left), genome_seq_left, Substring(genome_seq_right), genome_seq_right)
    if (vdiff * (Substring(genome_seq_left))) in genome_seq_left:
        print( v.CHROM, v.POS, v.REF, v.ALT[0])            
        failcount += 1
        print('vdiff in left')
    elif (vdiff * (Substring(genome_seq_right))) in genome_seq_right: 
        print( v.CHROM, v.POS, v.REF, v.ALT[0])
        failcount += 1
        print('vdiff in right')
    else:
        print( v.CHROM, v.POS, v.REF, v.ALT[0])
        passcount += 1
        print('PASS')
print(v.INFO.get('donor') + ":", 'Passcount is', passcount, 'and failcount is', failcount) 

 

8 CTAGTTTTTTTT 7 AAAAAAAGAAAT
chr1 75806922 T TA
vdiff in right
3 ACATCTGGGAAT 9 AAAAAAAAAGTG
chr2 39007201 T TA
vdiff in right
3 AAAGTGATTTAA 10 TTTTTTTTTTAA
chr2 39035331 AT A
vdiff in left
5 CCTTTTTCAGGT 12 AAAAAAAAAAAA
chr2 47414420 TAAA T
PASS
5 CCTTTTTCAGGT 12 AAAAAAAAAAAA
chr2 47414420 TAA T
PASS
3 AAAGTGTAAATT 3 AGGAAATGGGTT
chr2 47474913 T TAGG
PASS
4 TCATAAAAGACC 4 TTTTCCTCCCTC
chr2 47804889 CT C
vdiff in right
3 AGTATGCTTTAG 10 AAAAAAAAAATT
chr2 189852791 GA G
vdiff in right
2 GAAGCACAAGTT 5 AAAAACCTCATT
chr2 189863889 TA T
vdiff in left
3 TAGAAAGGATTT 8 AAAAAAAACATG
chr3 12609348 TA T
vdiff in left
2 GGATGAGGAGGT 3 AGAGCATGACCC
chr4 54731256 T TAG
PASS
3 AATGTTAGGGAC 8 AAAAAAAAGGGC
chr5 80675095 C CA
vdiff in right
3 AATAGCAGAAAT 4 AAAAGAAAAGAT
chr5 112839514 TAAAAG T
PASS
2 AACTTCACCAGT 4 AAAACCTATACC
chr5 112840921 TA T
vdiff in left
6 ATGCCAAAAAAG 5 AAAAAGCCTTCA
chr5 112841744 GA G
vdiff in left
5 AAAAATGTCATA 2 TACATCTCCAGG
chr5 112842691 A ATACATCTCCAGGTAGACAGATGAGCCAA

[W::vcf_parse] Contig 'chr1' is not defined in the header. (Quick workaround: index the file with tabix.)
[W::vcf_parse] FILTER '-' is not defined in the header
[W::vcf_parse] INFO 'donor' is not defined in the header, assuming Type=String
[W::vcf_parse] INFO 'variation_type' is not defined in the header, assuming Type=String
[W::vcf_parse] INFO 'subtype' is not defined in the header, assuming Type=String
[W::vcf_parse_format] FORMAT 'filter' is not defined in the header, assuming Type=String
[W::vcf_parse_format] FORMAT 'sample' is not defined in the header, assuming Type=String
[W::vcf_parse_format] FORMAT 'AF' is not defined in the header, assuming Type=String
[W::vcf_parse_format] FORMAT 'alt_depth' is not defined in the header, assuming Type=String
[W::vcf_parse_format] FORMAT 'depth' is not defined in the header, assuming Type=String
[W::vcf_parse_format] FORMAT 'no_calls' is not defined in the header, assuming Type=String
[W::vcf_parse_format] FORMAT 'tissue' is not defined in t